# Evaluate model accuracy

In this notebook I want to evaluate the accuracy of the best model obtained (i.e. `unet_v2.f1lo-b14-e60-lr0.001.44.hdf5`) using the old dataset as a test set.

### Load trained model

In [1]:
import tensorflow.keras.backend as K

def f1_score(y_true, y_pred):
    """F1 score metric.
    
    The F1 score is the harmonic mean of the precision and
    recall, where an F1 score reaches its best value at 1
    (perfect precision and recall).
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    p = true_positives / (predicted_positives + K.epsilon())
    r = true_positives / (possible_positives + K.epsilon())
    return 2 * p * r / (p + r + K.epsilon())

def f1_loss(y_true, y_pred):
    """
    The best loss function for F1-score would be, of course, the metric
    itself: minimizing 1 − F1 is same as maximizing F1.
    """
    true_positives = K.sum(y_true * y_pred)
    predicted_positives = K.sum(y_pred)
    possible_positives = K.sum(y_true)
    
    p = true_positives / (predicted_positives + K.epsilon())
    r = true_positives / (possible_positives + K.epsilon())
    return 1 - (2 * p * r / (p + r + K.epsilon()))

In [2]:
from tensorflow.keras.models import load_model

model = load_model(
    'models/unet_v2.f1lo-b14-e60-lr0.001.44.hdf5',
    custom_objects={'f1_score':f1_score, 'f1_loss': f1_loss}
)

### Load input images and masks

In [3]:
import glob

images = glob.glob('data/test/*_image.jpg')
images.sort()
masks = glob.glob('data/test/*_mask.jpg')
masks.sort()
print('Found %d images and %d masks' % (len(images), len(masks)))

Found 30 images and 30 masks


In [4]:
def process_images(images):
    """
    Converting RGB values from [0, 255] to [0, 1].
    """
    images /= 255.0
    return images.astype(np.float32)

def process_masks(masks):
    """
    Converting RGB values from [0, 255] to {0, 1}.
    """
    masks /= 255.0
    masks[masks > 0.5] = 1
    masks[masks <= 0.5] = 0
    return masks.astype(np.float32)

def process_results(predictions):
    """
    Processing values from [0, 1] to {0, 255}.
    The last dimension is removed.
    """
    predictions[predictions > 0.5] = 255
    predictions[predictions <= 0.5] = 0
    return predictions

In [5]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

x = []
for image in images:
    image = load_img(image)
    image = img_to_array(image)
    x.append(image)

x = np.array(x)
x = process_images(x)

y_true = []
for mask in masks:
    mask = load_img(mask)
    mask = img_to_array(mask)
    y_true.append(mask)

y_true = np.array(y_true)
y_true = process_masks(y_true)

### Predict masks

In [6]:
y_pred = model.predict(x)

### Calculate accuracy on test set

In [7]:
f1 = f1_score(y_true, y_pred)

print('F1-score: %f' % f1)

F1-score: 0.760333


### Save predicted masks

In [9]:
from tensorflow.keras.preprocessing.image import array_to_img

pred = process_results(y_pred)

for i in range(x.shape[0]):
    img = array_to_img(pred[i])
    img.save('data/test/%02d_pred.jpg' % i, quality=95)

### Visualize predicted masks

In [ ]:
import matplotlib.pyplot as plt

pred = process_results(y_pred)
pred = np.squeeze(pred, axis=-1)

fig, axes = plt.subplots(x.shape[0], 3, figsize=(20,200))
for i in range(x.shape[0]):
    axes[i][0].imshow(x[i])
    axes[i][0].set_title('Image %d' % i)
    axes[i][0].get_xaxis().set_visible(False)
    axes[i][0].get_yaxis().set_visible(False)

    axes[i][1].imshow(pred[i], cmap='gray', vmin=0, vmax=255)
    axes[i][1].set_title('Predicted mask')
    axes[i][1].get_xaxis().set_visible(False)
    axes[i][1].get_yaxis().set_visible(False)
    
    axes[i][2].imshow(y_true[i], cmap='gray', vmin=0, vmax=255)
    axes[i][2].set_title('Correct mask')
    axes[i][2].get_xaxis().set_visible(False)
    axes[i][2].get_yaxis().set_visible(False)

plt.subplots_adjust(wspace=0, hspace=0.1)